In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210718.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,14/07/2021 08:37:42,Murder on the Orient Express (1974),The Umbrellas of Cherbourg (1964),Umberto D (1952),INDIGENOUS,TRAIN WEEK,[Default to Nicolas Cage],IT WAS ALL DREAM(?)
1,14/07/2021 08:42:34,Umberto D (1952),Murder on the Orient Express (1974),NaN,IT WAS ALL DREAM(?),TRAIN WEEK,[Default to Nicolas Cage],INDIGENOUS
2,14/07/2021 08:47:08,The Umbrellas of Cherbourg (1964),Murder on the Orient Express (1974),Umberto D (1952),TRAIN WEEK,INDIGENOUS,[Default to Nicolas Cage],IT WAS ALL DREAM(?)
3,14/07/2021 09:26:41,The Umbrellas of Cherbourg (1964),Murder on the Orient Express (1974),Umberto D (1952),IT WAS ALL DREAM(?),TRAIN WEEK,INDIGENOUS,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Murder on the Orient Express (1974)',
       'The Umbrellas of Cherbourg (1964)', 'Umberto D (1952)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Murder on the Orient Express (1974)': <Candidate('Murder on the Orient Express (1974)')>,
 'The Umbrellas of Cherbourg (1964)': <Candidate('The Umbrellas of Cherbourg (1964)')>,
 'Umberto D (1952)': <Candidate('Umberto D (1952)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Murder on the Orient Express (1974), The Umbrellas of Cherbourg (1964), Umberto D (1952))>,
 <Ballot(Umberto D (1952), Murder on the Orient Express (1974))>,
 <Ballot(The Umbrellas of Cherbourg (1964), Murder on the Orient Express (1974), Umberto D (1952))>,
 <Ballot(The Umbrellas of Cherbourg (1964), Murder on the Orient Express (1974), Umberto D (1952))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                              Votes  Status
-----------------------------------  -------  --------
The Umbrellas of Cherbourg (1964)          2  Elected
Murder on the Orient Express (1974)        1  Rejected
Umberto D (1952)                           1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
IT WAS ALL DREAM(?)              2  Elected
TRAIN WEEK                       1  Rejected
INDIGENOUS                       1  Rejected
[Default to Nicolas Cage]        0  Rejected

